In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

from src.config import c as gc
from src.generator import Generator, default_image_augmenation_options
from src.utils import fix_random_seed

In [2]:
fix_random_seed()

In [3]:
BATCH = 96
IMAGE_SIZE = (224, 224)
AUGM_OPTS = default_image_augmenation_options
# AUGM_OPTS = None
CACHE_DIR = gc["DATA_DIR"] + "/images_cache"
# CACHE_DIR = None

In [4]:
gen = Generator(
    csv_file=f"{gc['DATA_DIR']}/src/train.csv",
    images_src_dir=f"{gc['DATA_DIR']}/src/train_images",
    batch_size=BATCH,
    target_image_size=IMAGE_SIZE,
    image_augmentation_options=AUGM_OPTS,
    cache_dir=CACHE_DIR,
    shuffle=True,
)

In [5]:
x0, y0 = gen.get_one(0)

In [6]:
base_model = keras.applications.ResNet50(
    weights="imagenet", include_top=False, input_shape=x0.shape, classes=y0.shape[0]
)

x = base_model.output
x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = keras.layers.Flatten(name="flatten")(x)
x = keras.layers.Dense(y0.shape[0], activation="sigmoid")(x)

model = keras.Model(inputs=base_model.inputs, outputs=[x])

In [7]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
model.fit(
    x=gen,
    epochs=10,
    verbose=1,
    max_queue_size=16,
    workers=16,
    use_multiprocessing=True,
)

Epoch 1/10
194/194 [==============================] - 79s 407ms/step - loss: 1.0064
Epoch 2/10


Process Keras_worker_ForkPoolWorker-24:
Process Keras_worker_ForkPoolWorker-29:
Process Keras_worker_ForkPoolWorker-23:
Process Keras_worker_ForkPoolWorker-31:
Process Keras_worker_ForkPoolWorker-47:
Process Keras_worker_ForkPoolWorker-35:
Process Keras_worker_ForkPoolWorker-18:
Process Keras_worker_ForkPoolWorker-26:
Process Keras_worker_ForkPoolWorker-32:
Process Keras_worker_ForkPoolWorker-42:
Process Keras_worker_ForkPoolWorker-38:
Process Keras_worker_ForkPoolWorker-17:
Process Keras_worker_ForkPoolWorker-33:
Process Keras_worker_ForkPoolWorker-21:
Process Keras_worker_ForkPoolWorker-41:
Process Keras_worker_ForkPoolWorker-28:
Process Keras_worker_ForkPoolWorker-25:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-27:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-19:
Process Keras_worker_ForkPoolWorker-43:
Process Keras_worker_ForkPoolWorker-48:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rece